# 挂载文件

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# KDD-UEBA

## 数据预处理

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


kddcup99 = pd.read_csv('/content/drive/MyDrive/kddcup99(raw).csv',encoding='gbk')
kddcup99.columns = kddcup99.columns.str.strip()
kddcup99.replace([np.inf, -np.inf], np.nan, inplace=True)
kddcup99.dropna(inplace=True)

def map_label_to_category(label):
    probe_attacks = ['ipsweep', 'nmap', 'portsweep']
    dos_attacks = ['smurf', 'neptune', 'back','teardrop','pod','land']
    u2r_attacks = ['buffer_overflow','rootkit','loadmodule','perl']
    r2l_attacks = ['ftp_write', 'imap', 'multihop','phf', 'spy', 'warezclient','warezmaster','guess_passwd']

    if label in probe_attacks:
        return 'Probe'
    elif label in dos_attacks:
        return 'DoS'
    elif label in u2r_attacks:
        return 'U2R'
    elif label in r2l_attacks:
        return 'R2L'
    elif label == 'normal':
        return 'BENIGN'


kddcup99['label'] = kddcup99['label'].apply(map_label_to_category)
# 检查新的类别分布
category_counts = kddcup99['label'].value_counts()
kddcup99['label'].value_counts()

values_list = ['BENIGN','Probe', 'DoS', 'U2R','R2L']
filtered_df = kddcup99[kddcup99['label'].isin(values_list)]


pd_train_v1 = pd.read_csv('/content/drive/MyDrive/CVUEBA.csv',encoding='gbk')
pd_train_v1 = pd_train_v1.rename(columns={'attack': 'label'})


def get_label(Label):
    if Label == 0:
        return 'Normal'
    else:
        return 'Malicious'

pd_train_v1['label'] = pd_train_v1['label'].apply(get_label)

all_columns = set(filtered_df.columns) | set(pd_train_v1.columns)
filtered_df_v1 = filtered_df.reindex(columns=all_columns)
df_train_v2 = pd_train_v1.reindex(columns=all_columns)
pd_final = pd.concat([filtered_df_v1, df_train_v2], ignore_index=True)

nb_class = len(list(pd_final['label'].value_counts()))
pd_mini = pd_final.sample(n=len(pd_final))

object_cols = pd_mini.select_dtypes(include=['object']).columns.tolist()
object_cols.remove('label')

cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value=-99))
cat_ohe_step = ('ohe', OrdinalEncoder()) #离散数据整数化处理
cat_steps = [cat_si_step, cat_ohe_step]

cat_pipe = Pipeline(cat_steps)
cat_cols = object_cols
cat_transformers = [('cat', cat_pipe, cat_cols)]

ct = ColumnTransformer(transformers=cat_transformers)


for column_name in cat_cols:
  pd_mini[column_name] = pd_mini[column_name].astype('str')
X_kdd_transformed = ct.fit_transform(pd_mini)
non_object_cols = pd_mini.select_dtypes(exclude=['object']).columns.tolist()
num_cols = non_object_cols
num_si_step = ('si', SimpleImputer(strategy='median'))
num_ss_step = ('ss', StandardScaler()) #均值-标准差归一化
num_steps = [num_si_step, num_ss_step]
num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, num_cols)]
ct = ColumnTransformer(transformers=num_transformers)
X_num_transformed = ct.fit_transform(pd_mini)

transformers = [('cat', cat_pipe, cat_cols),
                ('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=transformers)
datax = pd_mini.drop(columns=['label'])
x_all = ct.fit_transform(datax)
le = LabelEncoder()
y_all = le.fit_transform(pd_mini['label'])
y_all = y_all.reshape(-1,1)
data = np.concatenate((y_all.reshape(-1,1),x_all),axis=1)
c = []
#重采样
for i in range(len(pd_final)):
    if data[i,0] == 2:
        data = np.concatenate((data,np.tile(data[i,:],(100,1))))
    if data[i, 0] == 4:
        data = np.concatenate((data, np.tile(data[i, :], (30, 1))))
    if data[i, 0] == 5:
        data = np.concatenate((data, np.tile(data[i, :], (200, 1))))
    if data[i,0] == 6:
        data = np.concatenate((data,np.tile(data[i,:],(1000,1))))
print(np.unique(data[:,0], return_counts=True))
#转化为独热编码
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    one_hot_labels = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        one_hot_labels[i, labels[i]] = 1
    return one_hot_labels

num_classes = 7
one_hot_labels = one_hot_encode(data[:,0].astype(int), num_classes)

x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(data[:,1:], one_hot_labels,stratify=one_hot_labels, test_size=0.1, random_state=1)


(array([0., 1., 2., 3., 4., 5., 6.]), array([ 97277, 391458,   1717,  23681,  78058, 226326,  52052]))


In [ ]:
print(len(pd_final))

516129


## Tabnet

In [ ]:
y_train_1 = np.argmax(y_train_1, axis=1)
y_test_1 = np.argmax(y_test_1, axis=1)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
# 定义TabNet模型
tabnet_params = {
    'n_d': 8,
    'n_a': 8,
    'n_steps': 3,
    'gamma': 1.5,
    'n_independent': 2,
    'n_shared': 2,
    'momentum': 0.02,
    'lambda_sparse': 1e-3,
    'optimizer_fn': torch.optim.Adam,
    'optimizer_params': dict(lr=2e-2),
    'mask_type': 'sparsemax',
    'scheduler_params': {"gamma": 0.95,
                         "step_size": 10},
    'scheduler_fn': torch.optim.lr_scheduler.StepLR
}

X_train = torch.tensor(x_train_1, dtype=torch.float32)
X_test = torch.tensor(x_test_1, dtype=torch.float32)

if y_train_1.ndim == 2:
    y_train = torch.tensor(np.argmax(y_train_1, axis=1), dtype=torch.long)
    y_test = torch.tensor(np.argmax(y_test_1, axis=1), dtype=torch.long)
else:
    y_train = torch.tensor(y_train_1, dtype=torch.long)
    y_test = torch.tensor(y_test_1, dtype=torch.long)

clf = TabNetClassifier(**tabnet_params)

clf.fit(
    x_train_1,
    y_train,
    max_epochs=20,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.14707 |  0:00:32s
epoch 1  | loss: 0.03319 |  0:01:03s
epoch 2  | loss: 0.02864 |  0:01:33s
epoch 3  | loss: 0.02415 |  0:02:04s
epoch 4  | loss: 0.02483 |  0:02:35s
epoch 5  | loss: 0.02535 |  0:03:05s
epoch 6  | loss: 0.02545 |  0:03:36s
epoch 7  | loss: 0.03065 |  0:04:07s
epoch 8  | loss: 0.02434 |  0:04:37s
epoch 9  | loss: 0.02035 |  0:05:08s
epoch 10 | loss: 0.02167 |  0:05:39s
epoch 11 | loss: 0.02101 |  0:06:09s
epoch 12 | loss: 0.01908 |  0:06:40s
epoch 13 | loss: 0.01956 |  0:07:10s
epoch 14 | loss: 0.0179  |  0:07:41s
epoch 15 | loss: 0.01932 |  0:08:11s
epoch 16 | loss: 0.01718 |  0:08:42s
epoch 17 | loss: 0.017   |  0:09:13s
epoch 18 | loss: 0.016   |  0:09:43s
epoch 19 | loss: 0.0186  |  0:10:14s


## 分类报告

In [ ]:
if hasattr(y_pred, 'detach'):
    y_pred = y_pred.detach().cpu().numpy()
elif hasattr(y_pred, 'numpy'):
    y_pred = y_pred.numpy()

if hasattr(y_test, 'detach'):
    y_test = y_test.detach().cpu().numpy()
elif hasattr(y_test, 'numpy'):
    y_test = y_test.numpy()


In [ ]:
# 打印分类报告
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, confusion_matrix, classification_report, accuracy_score

In [ ]:
# 评估模型
y_pred = clf.predict(x_test_1)
accuracy = accuracy_score(y_test, y_pred)
print(f'Final Test Accuracy: {accuracy:.4f}')

# 将预测和真实标签直接映射回原始类别标签
y_test_original = le.inverse_transform(y_test)
y_pred_original = le.inverse_transform(y_pred)

print(classification_report(y_test_original, y_pred_original, digits=4))


# 预测
y_pred = clf.predict(x_test_1)

# 准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# 检测率（召回率）
detection_rate = recall_score(y_test, y_pred, average='macro')  # 使用 'macro' 或 'weighted' 平均
print(f'Detection Rate: {detection_rate:.4f}')

# 混淆矩阵
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

false_alarm_rate = []
false_negative_rate = []

for i in range(len(conf_matrix)):
    tp = conf_matrix[i, i]
    fp = conf_matrix[:, i].sum() - tp
    fn = conf_matrix[i, :].sum() - tp
    tn = conf_matrix.sum() - (tp + fp + fn)

    if fp + tn > 0:
        false_alarm_rate.append(fp / (fp + tn))
    if fn + tp > 0:
        false_negative_rate.append(fn / (fn + tp))


avg_false_alarm_rate = sum(false_alarm_rate) / len(false_alarm_rate)
avg_false_negative_rate = sum(false_negative_rate) / len(false_negative_rate)

print(f'False Alarm Rate: {avg_false_alarm_rate:.4f}')
print(f'False Negative Rate: {avg_false_negative_rate:.4f}')



Final Test Accuracy: 0.9949
              precision    recall  f1-score   support

      BENIGN     0.9983    0.9881    0.9932      9728
         DoS     0.9999    0.9998    0.9999     39146
   Malicious     0.9942    1.0000    0.9971       172
      Normal     1.0000    0.9996    0.9998      2368
       Probe     0.9988    0.9986    0.9987      7806
         R2L     0.9831    0.9990    0.9910     22632
         U2R     0.9941    0.9439    0.9684      5205

    accuracy                         0.9949     87057
   macro avg     0.9955    0.9899    0.9926     87057
weighted avg     0.9949    0.9949    0.9948     87057

Accuracy: 0.9949
Detection Rate: 0.9899
Confusion Matrix:
[[ 9612     5     0     0     9    95     7]
 [    5 39140     0     0     0     1     0]
 [    0     0   172     0     0     0     0]
 [    0     0     1  2367     0     0     0]
 [   11     0     0     0  7795     0     0]
 [    0     0     0     0     0 22610    22]
 [    0     0     0     0     0   292  4913]]
F

In [ ]:
print(le.classes_)

['BENIGN' 'DoS' 'Malicious' 'Normal' 'Probe' 'R2L' 'U2R']


# NSL-UEBA

## 数据预处理

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


NSL_KDD = pd.read_csv('/content/drive/MyDrive/NSL_KDD.csv',encoding='gbk')
NSL_KDD.columns = NSL_KDD.columns.str.strip()
NSL_KDD.replace([np.inf, -np.inf], np.nan, inplace=True)
NSL_KDD.dropna(inplace=True)

def map_label_to_category(label):
    probe_attacks = ['satan', 'ipsweep', 'nmap', 'portsweep', 'mscan', 'saint']
    dos_attacks = ['neptune', 'smurf', 'back', 'teardrop', 'pod', 'land', 'apache2', 'processtable', 'mailbomb', 'snmpgetattack', 'httptunnel', 'worm', 'sqlattack', 'udpstorm']
    u2r_attacks = ['buffer_overflow', 'loadmodule', 'rootkit', 'perl', 'xterm', 'ps']
    r2l_attacks = ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster', 'warezclient', 'spy', 'named', 'sendmail', 'snmpguess', 'xlock', 'xsnoop']


    if label in probe_attacks:
        return 'Probe'
    elif label in dos_attacks:
        return 'DoS'
    elif label in u2r_attacks:
        return 'U2R'
    elif label in r2l_attacks:
        return 'R2L'
    elif label == 'normal':
        return 'BENIGN'

NSL_KDD['label'] = NSL_KDD['labels'].apply(map_label_to_category)
# 检查新的类别分布
category_counts = NSL_KDD['label'].value_counts()
NSL_KDD['label'].value_counts()

values_list = ['BENIGN','Probe', 'DoS', 'U2R','R2L']
filtered_df = NSL_KDD[NSL_KDD['label'].isin(values_list)]

NSL_KDD['label'] = NSL_KDD['labels'].apply(map_label_to_category)

pd_train_v1 = pd.read_csv('/content/drive/MyDrive/CVUEBA.csv',encoding='gbk')
pd_train_v1 = pd_train_v1.rename(columns={'attack': 'label'})


def get_label(Label):
    if Label == 0:
        return 'Normal'
    else:
        return 'Malicious'

pd_train_v1['label'] = pd_train_v1['label'].apply(get_label)
all_columns = set(filtered_df.columns) | set(pd_train_v1.columns)
filtered_df_v1 = filtered_df.reindex(columns=all_columns)
df_train_v2 = pd_train_v1.reindex(columns=all_columns)
pd_final = pd.concat([filtered_df_v1, df_train_v2], ignore_index=True)

nb_class = len(list(pd_final['label'].value_counts()))
pd_mini = pd_final.sample(n=len(pd_final))

object_cols = pd_mini.select_dtypes(include=['object']).columns.tolist()
object_cols.remove('label')

cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value=-99))
cat_ohe_step = ('ohe', OrdinalEncoder()) #离散数据整数化处理
cat_steps = [cat_si_step, cat_ohe_step]

cat_pipe = Pipeline(cat_steps)
cat_cols = object_cols
cat_transformers = [('cat', cat_pipe, cat_cols)]

ct = ColumnTransformer(transformers=cat_transformers)


for column_name in cat_cols:
  pd_mini[column_name] = pd_mini[column_name].astype('str')
X_kdd_transformed = ct.fit_transform(pd_mini)
non_object_cols = pd_mini.select_dtypes(exclude=['object']).columns.tolist()
num_cols = non_object_cols
num_si_step = ('si', SimpleImputer(strategy='median'))
num_ss_step = ('ss', StandardScaler()) #均值-标准差归一化
num_steps = [num_si_step, num_ss_step]
num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, num_cols)]
ct = ColumnTransformer(transformers=num_transformers)
X_num_transformed = ct.fit_transform(pd_mini)

transformers = [('cat', cat_pipe, cat_cols),
                ('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=transformers)
datax = pd_mini.drop(columns=['label'])
x_all = ct.fit_transform(datax)
le = LabelEncoder()
y_all = le.fit_transform(pd_mini['label'])
y_all = y_all.reshape(-1,1)
data = np.concatenate((y_all.reshape(-1,1),x_all),axis=1)
c = []



In [ ]:
for i in range(len(pd_final)):
    if data[i,0] == 2:
        data = np.concatenate((data,np.tile(data[i,:],(100,1))))
    if data[i, 0] == 5:
        data = np.concatenate((data, np.tile(data[i, :], (30, 1))))
    if data[i,0] == 6:
        data = np.concatenate((data,np.tile(data[i,:],(80,1))))
print(np.unique(data[:,0], return_counts=True))

(array([0., 1., 2., 3., 4., 5., 6.]), array([ 77054,  53700,   1717,  23681,  14077, 110639,   9477]))


In [ ]:
print(len(x_all))

172215


In [ ]:
# 编码
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    one_hot_labels = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        one_hot_labels[i, labels[i]] = 1
    return one_hot_labels

num_classes = 7
one_hot_labels = one_hot_encode(data[:,0].astype(int), num_classes)


## 加载数据

In [ ]:
x_train_NSL_KDD, x_test_NSL_KDD, y_train_NSL_KDD, y_test_NSL_KDD = train_test_split(data[:,1:], one_hot_labels,stratify=one_hot_labels, test_size=0.2, random_state=12) # 固定了随机数种子导致分割的数据集是重复的

## Tabnet

In [ ]:
y_train_NSL_KDD = np.argmax(y_train_NSL_KDD, axis=1)
y_test_NSL_KDD = np.argmax(y_test_NSL_KDD, axis=1)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# 定义TabNet模型
tabnet_params = {
    'n_d': 8,
    'n_a': 8,
    'n_steps': 3,
    'gamma': 1.5,
    'n_independent': 2,
    'n_shared': 2,
    'momentum': 0.02,
    'lambda_sparse': 1e-3,
    'optimizer_fn': torch.optim.Adam,
    'optimizer_params': dict(lr=2e-2),
    'mask_type': 'sparsemax',
    'scheduler_params': {"gamma": 0.95,
                         "step_size": 10},
    'scheduler_fn': torch.optim.lr_scheduler.StepLR
}

X_train = torch.tensor(x_train_1, dtype=torch.float32)
X_test = torch.tensor(x_test_1, dtype=torch.float32)

if y_train_1.ndim == 2:
    y_train = torch.tensor(np.argmax(y_train_1, axis=1), dtype=torch.long)
    y_test = torch.tensor(np.argmax(y_test_1, axis=1), dtype=torch.long)
else:
    y_train = torch.tensor(y_train_1, dtype=torch.long)
    y_test = torch.tensor(y_test_1, dtype=torch.long)

clf = TabNetClassifier(**tabnet_params)

clf.fit(
    x_train_1,
    y_train,
    max_epochs=20,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.33167 |  0:00:08s
epoch 1  | loss: 0.04716 |  0:00:17s
epoch 2  | loss: 0.04822 |  0:00:25s
epoch 3  | loss: 0.02737 |  0:00:34s
epoch 4  | loss: 0.0148  |  0:00:43s
epoch 5  | loss: 0.01388 |  0:00:51s
epoch 6  | loss: 0.01439 |  0:01:00s
epoch 7  | loss: 0.01129 |  0:01:09s
epoch 8  | loss: 0.01062 |  0:01:17s
epoch 9  | loss: 0.01088 |  0:01:26s
epoch 10 | loss: 0.00942 |  0:01:35s
epoch 11 | loss: 0.01086 |  0:01:43s
epoch 12 | loss: 0.00759 |  0:01:52s
epoch 13 | loss: 0.00818 |  0:02:01s
epoch 14 | loss: 0.01221 |  0:02:09s
epoch 15 | loss: 0.00795 |  0:02:18s
epoch 16 | loss: 0.00654 |  0:02:27s
epoch 17 | loss: 0.00693 |  0:02:36s
epoch 18 | loss: 0.00803 |  0:02:44s
epoch 19 | loss: 0.00741 |  0:02:53s


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, confusion_matrix, classification_report, accuracy_score

In [ ]:
# 评估模型
y_pred = clf.predict(x_test_NSL_KDD)
accuracy = accuracy_score(y_test_NSL_KDD, y_pred)
print(f'Final Test Accuracy: {accuracy:.4f}')

# 将预测和真实标签直接映射回原始类别标签
y_test_original = le.inverse_transform(y_test)
y_pred_original = le.inverse_transform(y_pred)

print(classification_report(y_test_original, y_pred_original, digits=4))

# 预测
y_pred = clf.predict(x_test_NSL_KDD)

# 准确率
accuracy = accuracy_score(y_test_NSL_KDD, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# 检测率（召回率）
detection_rate = recall_score(y_test_NSL_KDD, y_pred, average='macro')  # 使用 'macro' 或 'weighted' 平均
print(f'Detection Rate: {detection_rate:.4f}')

# 混淆矩阵
conf_matrix = confusion_matrix(y_test_NSL_KDD, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

false_alarm_rate = []
false_negative_rate = []

for i in range(len(conf_matrix)):
    tp = conf_matrix[i, i]
    fp = conf_matrix[:, i].sum() - tp
    fn = conf_matrix[i, :].sum() - tp
    tn = conf_matrix.sum() - (tp + fp + fn)

    if fp + tn > 0:
        false_alarm_rate.append(fp / (fp + tn))
    if fn + tp > 0:
        false_negative_rate.append(fn / (fn + tp))

avg_false_alarm_rate = sum(false_alarm_rate) / len(false_alarm_rate)
avg_false_negative_rate = sum(false_negative_rate) / len(false_negative_rate)

print(f'False Alarm Rate: {avg_false_alarm_rate:.4f}')
print(f'False Negative Rate: {avg_false_negative_rate:.4f}')


Final Test Accuracy: 0.9986
              precision    recall  f1-score   support

      BENIGN     1.0000    0.9989    0.9994     15411
         DoS     0.9983    0.9966    0.9975     10740
   Malicious     0.9942    1.0000    0.9971       343
      Normal     1.0000    0.9996    0.9998      4736
       Probe     0.9989    0.9940    0.9964      2815
         R2L     0.9981    0.9995    0.9988     22128
         U2R     0.9911    1.0000    0.9955      1896

    accuracy                         0.9986     58069
   macro avg     0.9972    0.9984    0.9978     58069
weighted avg     0.9986    0.9986    0.9986     58069

Accuracy: 0.9986
Detection Rate: 0.9984
Confusion Matrix:
[[15394     0     0     0     0     0    17]
 [    0 10704     0     0     3    33     0]
 [    0     0   343     0     0     0     0]
 [    0     0     2  4734     0     0     0]
 [    0     7     0     0  2798    10     0]
 [    0    11     0     0     0 22117     0]
 [    0     0     0     0     0     0  1896]]
F

# 对比试验(NSL-UEBA)

## 数据预处理

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


NSL_KDD = pd.read_csv('/content/drive/MyDrive/NSL_KDD.csv',encoding='gbk')
NSL_KDD.columns = NSL_KDD.columns.str.strip()
NSL_KDD.replace([np.inf, -np.inf], np.nan, inplace=True)
NSL_KDD.dropna(inplace=True)

def map_label_to_category(label):
    probe_attacks = ['satan', 'ipsweep', 'nmap', 'portsweep', 'mscan', 'saint']
    dos_attacks = ['neptune', 'smurf', 'back', 'teardrop', 'pod', 'land', 'apache2', 'processtable', 'mailbomb', 'snmpgetattack', 'httptunnel', 'worm', 'sqlattack', 'udpstorm']
    u2r_attacks = ['buffer_overflow', 'loadmodule', 'rootkit', 'perl', 'xterm', 'ps']
    r2l_attacks = ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster', 'warezclient', 'spy', 'named', 'sendmail', 'snmpguess', 'xlock', 'xsnoop']

    if label in probe_attacks:
        return 'Probe'
    elif label in dos_attacks:
        return 'DoS'
    elif label in u2r_attacks:
        return 'U2R'
    elif label in r2l_attacks:
        return 'R2L'
    elif label == 'normal':
        return 'BENIGN'

NSL_KDD['label'] = NSL_KDD['labels'].apply(map_label_to_category)

category_counts = NSL_KDD['label'].value_counts()
NSL_KDD['label'].value_counts()

values_list = ['BENIGN','Probe', 'DoS', 'U2R','R2L']
filtered_df = NSL_KDD[NSL_KDD['label'].isin(values_list)]


NSL_KDD['label'] = NSL_KDD['labels'].apply(map_label_to_category)

pd_train_v1 = pd.read_csv('/content/drive/MyDrive/CVUEBA.csv',encoding='gbk')
pd_train_v1 = pd_train_v1.rename(columns={'attack': 'label'})


def get_label(Label):
    if Label == 0:
        return 'Normal'
    else:
        return 'Malicious'

pd_train_v1['label'] = pd_train_v1['label'].apply(get_label)
# pd_train_v1.drop(columns=['day'],inplace=True)

all_columns = set(filtered_df.columns) | set(pd_train_v1.columns)
filtered_df_v1 = filtered_df.reindex(columns=all_columns)
df_train_v2 = pd_train_v1.reindex(columns=all_columns)
pd_final = pd.concat([filtered_df_v1, df_train_v2], ignore_index=True)

nb_class = len(list(pd_final['label'].value_counts()))
pd_mini = pd_final.sample(n=len(pd_final))

object_cols = pd_mini.select_dtypes(include=['object']).columns.tolist()
object_cols.remove('label')

cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value=-99))
cat_ohe_step = ('ohe', OrdinalEncoder()) #离散数据整数化处理
cat_steps = [cat_si_step, cat_ohe_step]

cat_pipe = Pipeline(cat_steps)
cat_cols = object_cols
cat_transformers = [('cat', cat_pipe, cat_cols)]

ct = ColumnTransformer(transformers=cat_transformers)


for column_name in cat_cols:
  pd_mini[column_name] = pd_mini[column_name].astype('str')
X_kdd_transformed = ct.fit_transform(pd_mini)
non_object_cols = pd_mini.select_dtypes(exclude=['object']).columns.tolist()
num_cols = non_object_cols
num_si_step = ('si', SimpleImputer(strategy='median'))
num_ss_step = ('ss', StandardScaler()) #均值-标准差归一化
num_steps = [num_si_step, num_ss_step]
num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, num_cols)]
ct = ColumnTransformer(transformers=num_transformers)
X_num_transformed = ct.fit_transform(pd_mini)

transformers = [('cat', cat_pipe, cat_cols),
                ('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=transformers)
datax = pd_mini.drop(columns=['label'])
x_all = ct.fit_transform(datax)
le = LabelEncoder()
y_all = le.fit_transform(pd_mini['label'])
y_all = y_all.reshape(-1,1)
data = np.concatenate((y_all.reshape(-1,1),x_all),axis=1)
c = []

def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    one_hot_labels = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        one_hot_labels[i, labels[i]] = 1
    return one_hot_labels

num_classes = 7
one_hot_labels = one_hot_encode(data[:,0].astype(int), num_classes)

x_train_NSL_KDD, x_test_NSL_KDD, y_train_NSL_KDD, y_test_NSL_KDD = train_test_split(data[:,1:], one_hot_labels,stratify=one_hot_labels, test_size=0.2, random_state=12) # 固定了随机数种子导致分割的数据集是重复的

y_train_NSL_KDD = np.argmax(y_train_NSL_KDD, axis=1)
y_test_NSL_KDD = np.argmax(y_test_NSL_KDD, axis=1)


## 最近邻分类

In [ ]:
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score

# 训练分类器
centroid_classifier = NearestCentroid()
centroid_classifier.fit(x_train_NSL_KDD, y_train_NSL_KDD)

# 预测
y_pred = centroid_classifier.predict(x_test_NSL_KDD)

# 评估
print("准确率:", accuracy_score(y_test_NSL_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_NSL_KDD, y_pred, digits=4))

accuracy = accuracy_score(y_test_NSL_KDD, y_pred)
print(f"准确率: {accuracy:.4f}")

准确率: 0.6629794152658015
分类报告:
              precision    recall  f1-score   support

           0     0.7119    0.8139    0.7595     15411
           1     0.6120    0.5338    0.5702     10740
           2     0.0028    0.7500    0.0057         4
           3     0.9996    0.5781    0.7326      4736
           4     0.7355    0.4860    0.5852      2815
           5     0.6121    0.6078    0.6100       714
           6     0.0146    0.6957    0.0287        23

    accuracy                         0.6630     34443
   macro avg     0.5270    0.6379    0.4703     34443
weighted avg     0.7196    0.6630    0.6789     34443

准确率: 0.6630


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_nearest_centroid.py:244: UserWarning: self.within_class_std_dev_ has at least 1 zero standard deviation.Inputs within the same classes for at least 1 feature are identical.
  warnings.warn(


## Logistic 回归

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 训练逻辑回归模型
lr_classifier = LogisticRegression(max_iter=200)
lr_classifier.fit(x_train_NSL_KDD, y_train_NSL_KDD)

# 预测
y_pred = lr_classifier.predict(x_test_NSL_KDD)

# 评估
print("准确率:", accuracy_score(y_test_NSL_KDD, y_pred))


print("分类报告:")
print(classification_report(y_test_NSL_KDD, y_pred, digits=4))

准确率: 0.8716139709084575
准确率: 0.8716
分类报告:
              precision    recall  f1-score   support

           0     0.8667    0.9312    0.8978     15411
           1     0.9254    0.8986    0.9118     10740
           2     0.0000    0.0000    0.0000         4
           3     0.9020    0.8587    0.8798      4736
           4     0.7207    0.6259    0.6700      2815
           5     0.3780    0.2647    0.3114       714
           6     1.0000    0.0435    0.0833        23

    accuracy                         0.8716     34443
   macro avg     0.6847    0.5175    0.5363     34443
weighted avg     0.8678    0.8716    0.8683     34443



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

## 朴素贝叶斯

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# 训练朴素贝叶斯模型
nb_classifier = GaussianNB()
nb_classifier.fit(x_train_NSL_KDD, y_train_NSL_KDD)

# 预测
y_pred = nb_classifier.predict(x_test_NSL_KDD)

# 评估
print("准确率:", accuracy_score(y_test_NSL_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_NSL_KDD, y_pred, digits=4))


准确率: 0.7062973608570682
分类报告:
              precision    recall  f1-score   support

           0     1.0000    0.5404    0.7016     15411
           1     0.7749    0.7555    0.7651     10740
           2     0.1176    1.0000    0.2105         4
           3     1.0000    0.9937    0.9968      4736
           4     0.3315    0.9861    0.4962      2815
           5     0.2279    0.5336    0.3194       714
           6     0.0210    0.7826    0.0409        23

    accuracy                         0.7063     34443
   macro avg     0.4961    0.7988    0.5044     34443
weighted avg     0.8584    0.7063    0.7368     34443



## 决策树

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_classifier = DecisionTreeClassifier(max_depth=1)
tree_classifier.fit(x_train_NSL_KDD, y_train_NSL_KDD)

# 预测
y_pred = tree_classifier.predict(x_test_NSL_KDD)

# 评估
print("准确率:", accuracy_score(y_test_NSL_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_NSL_KDD, y_pred, digits=4))


准确率: 0.7288563713962198
分类报告:
              precision    recall  f1-score   support

           0     0.8327    1.0000    0.9087     15411
           1     0.6082    0.9025    0.7267     10740
           2     0.0000    0.0000    0.0000         4
           3     0.0000    0.0000    0.0000      4736
           4     0.0000    0.0000    0.0000      2815
           5     0.0000    0.0000    0.0000       714
           6     0.0000    0.0000    0.0000        23

    accuracy                         0.7289     34443
   macro avg     0.2059    0.2718    0.2336     34443
weighted avg     0.5622    0.7289    0.6332     34443



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 消融实验

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score

# 定义TabNet模型
tabnet_params = {
    'n_d': 8,
    'n_a': 8,
    'n_steps': 3,
    'gamma': 1.5,
    'n_independent': 2,
    'n_shared': 2,
    'momentum': 0.02,
    'lambda_sparse': 1e-3,
    'optimizer_fn': torch.optim.Adam,
    'optimizer_params': dict(lr=2e-2),
    'mask_type': 'sparsemax',
    'scheduler_params': {"gamma": 0.95, 'step_size': 10},
    'scheduler_fn': torch.optim.lr_scheduler.StepLR
}

X_train = torch.tensor(x_train_NSL_KDD, dtype=torch.float32)
X_test = torch.tensor(x_test_NSL_KDD, dtype=torch.float32)

if y_train_NSL_KDD.ndim == 2:
    y_train = torch.tensor(np.argmax(y_train_NSL_KDD, axis=1), dtype=torch.long)
    y_test = torch.tensor(np.argmax(y_test_NSL_KDD, axis=1), dtype=torch.long)
else:
    y_train = torch.tensor(y_train_NSL_KDD, dtype=torch.long)
    y_test = torch.tensor(y_test_NSL_KDD, dtype=torch.long)

# 初始化TabNet分类器
clf = TabNetClassifier(**tabnet_params)

# 训练TabNet模型
clf.fit(
    X_train=X_train.numpy(),
    y_train=y_train.numpy(),
    max_epochs=20,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.3072  |  0:00:05s
epoch 1  | loss: 0.04735 |  0:00:10s
epoch 2  | loss: 0.03519 |  0:00:15s
epoch 3  | loss: 0.02322 |  0:00:21s
epoch 4  | loss: 0.02021 |  0:00:27s
epoch 5  | loss: 0.01239 |  0:00:34s
epoch 6  | loss: 0.01727 |  0:00:40s
epoch 7  | loss: 0.01762 |  0:00:45s
epoch 8  | loss: 0.01343 |  0:00:51s
epoch 9  | loss: 0.00971 |  0:00:56s
epoch 10 | loss: 0.00864 |  0:01:02s
epoch 11 | loss: 0.00737 |  0:01:07s
epoch 12 | loss: 0.0065  |  0:01:13s
epoch 13 | loss: 0.00649 |  0:01:19s
epoch 14 | loss: 0.00601 |  0:01:24s
epoch 15 | loss: 0.00559 |  0:01:29s
epoch 16 | loss: 0.00777 |  0:01:35s
epoch 17 | loss: 0.00811 |  0:01:40s
epoch 18 | loss: 0.00677 |  0:01:45s
epoch 19 | loss: 0.00812 |  0:01:50s


In [ ]:
# 评估模型
y_pred = clf.predict(x_test_NSL_KDD)
accuracy = accuracy_score(y_test_NSL_KDD, y_pred)
print(f'Final Test Accuracy: {accuracy:.4f}')

y_test_original = le.inverse_transform(y_test)
y_pred_original = le.inverse_transform(y_pred)


from sklearn.metrics import recall_score, confusion_matrix, classification_report, accuracy_score

# 预测
y_pred = clf.predict(x_test_NSL_KDD)

accuracy = accuracy_score(y_test_NSL_KDD, y_pred)
print(f'Accuracy: {accuracy:.4f}')
detection_rate = recall_score(y_test_NSL_KDD, y_pred, average='macro')  # 使用 'macro' 或 'weighted' 平均
print(f'Detection Rate: {detection_rate:.4f}')
conf_matrix = confusion_matrix(y_test_NSL_KDD, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

false_alarm_rate = []
false_negative_rate = []

for i in range(len(conf_matrix)):
    tp = conf_matrix[i, i]
    fp = conf_matrix[:, i].sum() - tp
    fn = conf_matrix[i, :].sum() - tp
    tn = conf_matrix.sum() - (tp + fp + fn)

    if fp + tn > 0:
        false_alarm_rate.append(fp / (fp + tn))
    if fn + tp > 0:
        false_negative_rate.append(fn / (fn + tp))

avg_false_alarm_rate = sum(false_alarm_rate) / len(false_alarm_rate)
avg_false_negative_rate = sum(false_negative_rate) / len(false_negative_rate)

print(f'False Alarm Rate: {avg_false_alarm_rate:.4f}')
print(f'False Negative Rate: {avg_false_negative_rate:.4f}')


Final Test Accuracy: 0.9981
Accuracy: 0.9981
Detection Rate: 0.7675
Confusion Matrix:
[[15411     0     0     0     0     0     0]
 [    0 10711     0     0     3    26     0]
 [    0     0     0     4     0     0     0]
 [    0     0     0  4736     0     0     0]
 [    0    10     0     0  2805     0     0]
 [    0     3     0     0     5   705     1]
 [    0     4     0     0     5     5     9]]
False Alarm Rate: 0.0003
False Negative Rate: 0.2325


# 对比试验(KDD-UEBA)

## 数据预处理

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


NSL_KDD = pd.read_csv('/content/drive/MyDrive/kddcup99(raw).csv',encoding='gbk')
NSL_KDD.columns = NSL_KDD.columns.str.strip()
NSL_KDD.replace([np.inf, -np.inf], np.nan, inplace=True)
NSL_KDD.dropna(inplace=True)
def map_label_to_category(label):
    probe_attacks = ['satan', 'ipsweep', 'nmap', 'portsweep', 'mscan', 'saint']
    dos_attacks = ['neptune', 'smurf', 'back', 'teardrop', 'pod', 'land', 'apache2', 'processtable', 'mailbomb', 'snmpgetattack', 'httptunnel', 'worm', 'sqlattack', 'udpstorm']
    u2r_attacks = ['buffer_overflow', 'loadmodule', 'rootkit', 'perl', 'xterm', 'ps']
    r2l_attacks = ['guess_passwd', 'ftp_write', 'imap', 'phf', 'multihop', 'warezmaster', 'warezclient', 'spy', 'named', 'sendmail', 'snmpguess', 'xlock', 'xsnoop']

    if label in probe_attacks:
        return 'Probe'
    elif label in dos_attacks:
        return 'DoS'
    elif label in u2r_attacks:
        return 'U2R'
    elif label in r2l_attacks:
        return 'R2L'
    elif label == 'normal':
        return 'BENIGN'


NSL_KDD['label'] = NSL_KDD['label'].apply(map_label_to_category)
category_counts = NSL_KDD['label'].value_counts()
NSL_KDD['label'].value_counts()

values_list = ['BENIGN','Probe', 'DoS', 'U2R','R2L']
filtered_df = NSL_KDD[NSL_KDD['label'].isin(values_list)]

NSL_KDD['label'] = NSL_KDD['label'].apply(map_label_to_category)

pd_train_v1 = pd.read_csv('/content/drive/MyDrive/CVUEBA.csv',encoding='gbk')
pd_train_v1 = pd_train_v1.rename(columns={'attack': 'label'})


def get_label(Label):
    if Label == 0:
        return 'Normal'
    else:
        return 'Malicious'

pd_train_v1['label'] = pd_train_v1['label'].apply(get_label)
# pd_train_v1.drop(columns=['day'],inplace=True)

all_columns = set(filtered_df.columns) | set(pd_train_v1.columns)
filtered_df_v1 = filtered_df.reindex(columns=all_columns)
df_train_v2 = pd_train_v1.reindex(columns=all_columns)
pd_final = pd.concat([filtered_df_v1, df_train_v2], ignore_index=True)

nb_class = len(list(pd_final['label'].value_counts()))
pd_mini = pd_final.sample(n=len(pd_final))

object_cols = pd_mini.select_dtypes(include=['object']).columns.tolist()
object_cols.remove('label')

cat_si_step = ('si', SimpleImputer(strategy='constant', fill_value=-99))
cat_ohe_step = ('ohe', OrdinalEncoder()) #离散数据整数化处理
cat_steps = [cat_si_step, cat_ohe_step]

cat_pipe = Pipeline(cat_steps)
cat_cols = object_cols
cat_transformers = [('cat', cat_pipe, cat_cols)]

ct = ColumnTransformer(transformers=cat_transformers)


for column_name in cat_cols:
  pd_mini[column_name] = pd_mini[column_name].astype('str')
X_kdd_transformed = ct.fit_transform(pd_mini)
non_object_cols = pd_mini.select_dtypes(exclude=['object']).columns.tolist()
num_cols = non_object_cols
num_si_step = ('si', SimpleImputer(strategy='median'))
num_ss_step = ('ss', StandardScaler()) #均值-标准差归一化
num_steps = [num_si_step, num_ss_step]
num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, num_cols)]
ct = ColumnTransformer(transformers=num_transformers)
X_num_transformed = ct.fit_transform(pd_mini)

transformers = [('cat', cat_pipe, cat_cols),
                ('num', num_pipe, num_cols)]

ct = ColumnTransformer(transformers=transformers)
datax = pd_mini.drop(columns=['label'])
x_all = ct.fit_transform(datax)
le = LabelEncoder()
y_all = le.fit_transform(pd_mini['label'])
y_all = y_all.reshape(-1,1)
data = np.concatenate((y_all.reshape(-1,1),x_all),axis=1)
c = []

#转化为独热编码
def one_hot_encode(labels, num_classes):
    num_samples = len(labels)
    one_hot_labels = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        one_hot_labels[i, labels[i]] = 1
    return one_hot_labels

num_classes = 7
one_hot_labels = one_hot_encode(data[:,0].astype(int), num_classes)

x_train_KDD, x_test_KDD, y_train_KDD, y_test_KDD = train_test_split(data[:,1:], one_hot_labels,stratify=one_hot_labels, test_size=0.2, random_state=12) # 固定了随机数种子导致分割的数据集是重复的


In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
X_train = torch.tensor(x_train_KDD, dtype=torch.float32)
X_test = torch.tensor(x_test_KDD, dtype=torch.float32)

if y_train_KDD.ndim == 2:
    y_train = torch.tensor(np.argmax(y_train_KDD, axis=1), dtype=torch.long)
    y_test = torch.tensor(np.argmax(y_test_KDD, axis=1), dtype=torch.long)
else:
    y_train = torch.tensor(y_train_KDD, dtype=torch.long)
    y_test = torch.tensor(y_test_KDD, dtype=torch.long)


In [ ]:
y_train_KDD = np.argmax(y_train_KDD, axis=1)
y_test_KDD = np.argmax(y_test_KDD, axis=1)


## 最近邻分类（Nearest Centroid Classifier）

In [ ]:
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score

# 训练最近邻分类器
centroid_classifier = NearestCentroid()
centroid_classifier.fit(x_train_KDD, y_train_KDD)

# 预测
y_pred = centroid_classifier.predict(x_test_KDD)

# 评估
print("准确率:", accuracy_score(y_test_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_KDD, y_pred, digits=4))

accuracy = accuracy_score(y_test_KDD, y_pred)
print(f"准确率: {accuracy:.4f}")

准确率: 0.6454840454299622
分类报告:
              precision    recall  f1-score   support

           0     0.8239    0.2600    0.3952     19456
           1     0.8524    0.7322    0.7878     78292
           2     0.6667    0.6667    0.6667         3
           3     0.9997    0.8055    0.8922      4736
           4     0.0214    0.6800    0.0414       822
           5     0.4776    0.2844    0.3565       225
           6     0.3000    0.9000    0.4500        10

    accuracy                         0.6455    103544
   macro avg     0.5917    0.6184    0.5128    103544
weighted avg     0.8463    0.6455    0.7119    103544

准确率: 0.6455


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_nearest_centroid.py:244: UserWarning: self.within_class_std_dev_ has at least 1 zero standard deviation.Inputs within the same classes for at least 1 feature are identical.
  warnings.warn(


## Logistic 回归（Logistic Regression）

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 训练逻辑回归模型
lr_classifier = LogisticRegression(max_iter=200)
lr_classifier.fit(x_train_KDD, y_train_KDD)

# 预测
y_pred = lr_classifier.predict(x_test_KDD)

# 评估
print("准确率:", accuracy_score(y_test_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_KDD, y_pred, digits=4))

准确率: 0.9860252646218033
分类报告:
              precision    recall  f1-score   support

           0     0.9529    0.9814    0.9669     19456
           1     0.9958    0.9940    0.9949     78292
           2     0.0000    0.0000    0.0000         3
           3     0.9748    0.9799    0.9774      4736
           4     0.9312    0.5925    0.7242       822
           5     0.7260    0.2356    0.3557       225
           6     0.0000    0.0000    0.0000        10

    accuracy                         0.9860    103544
   macro avg     0.6544    0.5405    0.5742    103544
weighted avg     0.9856    0.9860    0.9852    103544



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

## 朴素贝叶斯（Naive Bayes）

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# 训练朴素贝叶斯模型
nb_classifier = GaussianNB()
nb_classifier.fit(x_train_KDD, y_train_KDD)

# 预测
y_pred = nb_classifier.predict(x_test_KDD)

# 评估
print("准确率:", accuracy_score(y_test_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_KDD, y_pred, digits=4))


准确率: 0.8866472224368385
分类报告:
              precision    recall  f1-score   support

           0     0.9659    0.6443    0.7730     19456
           1     0.9769    0.9407    0.9584     78292
           2     0.1500    1.0000    0.2609         3
           3     1.0000    0.9964    0.9982      4736
           4     0.0924    0.9830    0.1689       822
           5     0.3772    0.3822    0.3797       225
           6     0.0062    0.9000    0.0122        10

    accuracy                         0.8866    103544
   macro avg     0.5098    0.8352    0.5073    103544
weighted avg     0.9674    0.8866    0.9178    103544



## 决策树（DT）

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_classifier = DecisionTreeClassifier(max_depth=1)
tree_classifier.fit(x_train_KDD, y_train_KDD)

# 预测
y_pred = tree_classifier.predict(x_test_KDD)

# 评估
print("准确率:", accuracy_score(y_test_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_KDD, y_pred, digits=4))


准确率: 0.9321158154987251
分类报告:
              precision    recall  f1-score   support

           0     0.9178    0.9863    0.9508     19456
           1     0.9357    0.9876    0.9610     78292
           2     0.0000    0.0000    0.0000         3
           3     0.0000    0.0000    0.0000      4736
           4     0.0000    0.0000    0.0000       822
           5     0.0000    0.0000    0.0000       225
           6     0.0000    0.0000    0.0000        10

    accuracy                         0.9321    103544
   macro avg     0.2648    0.2820    0.2731    103544
weighted avg     0.8800    0.9321    0.9053    103544



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# 训练朴素贝叶斯模型
nb_classifier = GaussianNB()
nb_classifier.fit(x_train_KDD, y_train_KDD)

# 预测
y_pred = nb_classifier.predict(x_test_KDD)

# 评估
print("准确率:", accuracy_score(y_test_KDD, y_pred))
print("分类报告:")
print(classification_report(y_test_KDD, y_pred, digits=4))


准确率: 0.8866472224368385
分类报告:
              precision    recall  f1-score   support

           0     0.9659    0.6443    0.7730     19456
           1     0.9769    0.9407    0.9584     78292
           2     0.1500    1.0000    0.2609         3
           3     1.0000    0.9964    0.9982      4736
           4     0.0924    0.9830    0.1689       822
           5     0.3772    0.3822    0.3797       225
           6     0.0062    0.9000    0.0122        10

    accuracy                         0.8866    103544
   macro avg     0.5098    0.8352    0.5073    103544
weighted avg     0.9674    0.8866    0.9178    103544



## 消融实验

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score

# 定义TabNet模型
tabnet_params = {
    'n_d': 8,
    'n_a': 8,
    'n_steps': 3,
    'gamma': 1.5,
    'n_independent': 2,
    'n_shared': 2,
    'momentum': 0.02,
    'lambda_sparse': 1e-3,
    'optimizer_fn': torch.optim.Adam,
    'optimizer_params': dict(lr=2e-2),
    'mask_type': 'sparsemax',
    'scheduler_params': {"gamma": 0.95, 'step_size': 10},
    'scheduler_fn': torch.optim.lr_scheduler.StepLR
}

X_train = torch.tensor(x_train_KDD, dtype=torch.float32)
X_test = torch.tensor(x_test_KDD, dtype=torch.float32)
if y_train_KDD.ndim == 2:
    y_train = torch.tensor(np.argmax(y_train_KDD, axis=1), dtype=torch.long)
    y_test = torch.tensor(np.argmax(y_test_KDD, axis=1), dtype=torch.long)
else:
    y_train = torch.tensor(y_train_KDD, dtype=torch.long)
    y_test = torch.tensor(y_test_KDD, dtype=torch.long)

# 初始化TabNet分类器
clf = TabNetClassifier(**tabnet_params)

# 训练TabNet模型
clf.fit(
    X_train=X_train.numpy(),
    y_train=y_train.numpy(),
    max_epochs=20,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.09425 |  0:00:15s
epoch 1  | loss: 0.01664 |  0:00:31s
epoch 2  | loss: 0.01301 |  0:00:46s
epoch 3  | loss: 0.00962 |  0:01:02s
epoch 4  | loss: 0.00912 |  0:01:17s
epoch 5  | loss: 0.00793 |  0:01:33s
epoch 6  | loss: 0.0074  |  0:01:48s
epoch 7  | loss: 0.00721 |  0:02:04s
epoch 8  | loss: 0.00817 |  0:02:20s
epoch 9  | loss: 0.01094 |  0:02:36s
epoch 10 | loss: 0.00922 |  0:02:51s
epoch 11 | loss: 0.0078  |  0:03:07s
epoch 12 | loss: 0.00949 |  0:03:22s
epoch 13 | loss: 0.00725 |  0:03:38s
epoch 14 | loss: 0.0073  |  0:03:53s
epoch 15 | loss: 0.00676 |  0:04:09s
epoch 16 | loss: 0.00674 |  0:04:24s
epoch 17 | loss: 0.00632 |  0:04:40s
epoch 18 | loss: 0.00563 |  0:04:56s
epoch 19 | loss: 0.00938 |  0:05:11s


In [ ]:
# 评估模型
y_pred = clf.predict(x_test_KDD)
accuracy = accuracy_score(y_test_KDD, y_pred)
print(f'Final Test Accuracy: {accuracy:.4f}')

# 将预测和真实标签直接映射回原始类别标签
y_test_original = le.inverse_transform(y_test)
y_pred_original = le.inverse_transform(y_pred)


from sklearn.metrics import recall_score, confusion_matrix, classification_report, accuracy_score

# 预测
y_pred = clf.predict(x_test_KDD)

accuracy = accuracy_score(y_test_KDD, y_pred)
print(f'Accuracy: {accuracy:.4f}')

detection_rate = recall_score(y_test_KDD, y_pred, average='macro')  # 使用 'macro' 或 'weighted' 平均
print(f'Detection Rate: {detection_rate:.4f}')

conf_matrix = confusion_matrix(y_test_KDD, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

false_alarm_rate = []
false_negative_rate = []

for i in range(len(conf_matrix)):
    tp = conf_matrix[i, i]
    fp = conf_matrix[:, i].sum() - tp
    fn = conf_matrix[i, :].sum() - tp
    tn = conf_matrix.sum() - (tp + fp + fn)

    if fp + tn > 0:
        false_alarm_rate.append(fp / (fp + tn))
    if fn + tp > 0:
        false_negative_rate.append(fn / (fn + tp))

avg_false_alarm_rate = sum(false_alarm_rate) / len(false_alarm_rate)
avg_false_negative_rate = sum(false_negative_rate) / len(false_negative_rate)

print(f'False Alarm Rate: {avg_false_alarm_rate:.4f}')
print(f'False Negative Rate: {avg_false_negative_rate:.4f}')


Final Test Accuracy: 0.9946
Accuracy: 0.9946
Detection Rate: 0.8313
Confusion Matrix:
[[19401    14     0     0     7    34     0]
 [  433 77858     0     1     0     0     0]
 [    0     0     3     0     0     0     0]
 [    0     0     4  4732     0     0     0]
 [   14     2     0     0   805     1     0]
 [   29     3     0     0     2   191     0]
 [    1     0     0     0     0     9     0]]
False Alarm Rate: 0.0010
False Negative Rate: 0.1687
